# Embeddings Part 3
# Going Through the Use Cases
Here we show some representative use cases. We will use the Amazon fine-food reviews dataset for the following examples.


In [ ]:
# Standard library imports
import os
import pickle
import re

# Related third party imports
import numpy as np
import pandas as pd
from scipy.spatial import distance
from ast import literal_eval

# Local application/library specific imports
import tiktoken
from openai import OpenAI


In [ ]:
# Create an instance of the OpenAI client
# This assumes you have set an environment variable called OPENAI_API_KEY
client = OpenAI()

First, let's see what trying to answer without using RAG looks like

In [ ]:
from openai import OpenAI

# an example question about the Amazon reviews
query = "What people hated their food?"

response = client.chat.completions.create(
    messages=[
        {'role': 'system', 'content': 'You answer questions about Amazon fine food reviews.'},
        {'role': 'user', 'content': query},
    ],
    model='gpt-4-turbo',
    temperature=0,
)

print(response.choices[0].message.content)

## Obtaining the Embeddings
The dataset contains a total of 568,454 food reviews Amazon users left up to October 2012. We will use a subset of 1,000 most recent reviews for illustration purposes. The reviews are in English and tend to be positive or negative. Each review has a ProductId, UserId, Score, review title (Summary) and review body (Text). For example:

| PRODUCT ID  | USER ID        | SCORE | SUMMARY             | TEXT                                           |
|-------------|----------------|-------|---------------------|------------------------------------------------|
| B001E4KFG0  | A3SGXH7AUHU8GW | 5     | Good Quality Dog Food | I have bought several of the Vitality canned... |
| B00813GRG4  | A1D87F6ZCVE5NK | 1     | Not as Advertised   | Product arrived labeled as Jumbo Salted Peanut... |



We will combine the review summary and review text into a single combined text. The model will encode this combined text and output a single vector embedding.

In [ ]:
# utility function to get the file size
def get_file_size(file_path):
    """ Returns the size of the file in megabytes. """
    size_bytes = os.path.getsize(file_path)
    size_mb = size_bytes / (1024 * 1024)  # Convert from bytes to megabytes
    return size_mb

# main utility function to get the embeddings
def get_embedding(text, model="text-embedding-3-small"):
    # Replace newlines in the text with spaces for consistent formatting
    text = text.replace("\n", " ")
    # Request the embedding for the cleaned text and return the embedding
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# utility function to get the embeddings with reduced dimensions
def get_embedding_reduced_dims(text, model="text-embedding-3-large"):
    # Replace newlines in the text with spaces for consistent formatting
    text = text.replace("\n", " ")
    # Request the embedding for the cleaned text and return the embedding
    return client.embeddings.create(input=[text], model=model,dimensions=1024).data[0].embedding

# utility function to get the embeddings with reduced dimensions
def get_embedding_large(text, model="text-embedding-3-large"):
    # Replace newlines in the text with spaces for consistent formatting
    text = text.replace("\n", " ")
    # Request the embedding for the cleaned text and return the embedding
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [ ]:

# Define the path to the input data file
input_data_path = "./EmbeddingsDemoAssets/fine_food_reviews_1k.csv"

# Read the CSV file using pandas and set the first column as the index
df = pd.read_csv(input_data_path, index_col=0)

# Select only the relevant columns from the dataframe
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]

# Drop rows with any missing values
df = df.dropna()

# Drop rows with any missing values
df = df.drop_duplicates()

# Combine the 'Summary' and 'Text' columns into a new column with a formatted string
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)

# Display the first 5 rows of the modified dataframe
df.head(5)


### Small Model Embedding
Now let's embed using the small model first

In [ ]:

# Apply the `get_embedding` function to each entry in the 'combined' column and store the results
df['embedding'] = df['combined'].apply(lambda x: get_embedding(x))

# Save the dataframe with embeddings to a CSV file, omitting the index
df.to_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_1k.csv', index=False)

# Display the first 5 rows of the modified dataframe
df.head(5)


In [ ]:
# Get the size of the data file
file_path = './EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_1k.csv'
size_mb = get_file_size(file_path)
print(f"The size of the file is {size_mb:.2f} MB.")

### Large Model Embedding
Let's embed using the large model next.

In [ ]:

# Apply the `get_embedding_large` function to each entry in the 'combined' column and store the results
df['embedding'] = df['combined'].apply(lambda x: get_embedding_large(x))

# Save the dataframe with embeddings to a CSV file, omitting the index
df.to_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_large_1k.csv', index=False)

# Display the first 5 rows of the modified dataframe
df.head(5)

In [ ]:
# Get the size of the data file
file_path = './EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_large_1k.csv'
size_mb = get_file_size(file_path)
print(f"The size of the file is {size_mb:.2f} MB.")

### Loading the Data
Here we show how to load the data into a dataframe from a CSV file to make it ready to be used again when needed. 

In [ ]:

# Load the CSV file into a pandas DataFrame
df_load = pd.read_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_large_1k.csv')

# Convert the string representation of embeddings in the 'embedding' column to numpy arrays
# By converting the embeddings from string format to numpy arrays immediately after loading, 
# we ensure that the data is in a ready-to-use state for any subsequent analysis or processing steps.
df_load['embedding'] = df_load['embedding'].apply(eval).apply(np.array)

# Display the first 5 rows of the loaded dataframe
df_load.head(5)

### Reducing the Dimensions
Using larger embeddings, for example storing them in a vector store for retrieval, generally costs more and consumes more compute, memory and storage than using smaller embeddings.

Both of our new embedding models were trained with a technique that allows developers to trade-off performance and cost of using embeddings. Specifically, developers can shorten embeddings (i.e. remove some numbers from the end of the sequence) without the embedding losing its concept-representing properties.

In [ ]:

# Define the path to the input data file
input_data_path = "./EmbeddingsDemoAssets/fine_food_reviews_1k.csv"

# Read the CSV file using pandas and set the first column as the index
df_reduced_dims = pd.read_csv(input_data_path, index_col=0)

# Select only the relevant columns from the dataframe
df_reduced_dims = df_reduced_dims[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]

# Drop rows with any missing values
df_reduced_dims = df_reduced_dims.dropna()

# Drop rows with any missing values
df_reduced_dims = df_reduced_dims.drop_duplicates()

# Combine the 'Summary' and 'Text' columns into a new column with a formatted string
df_reduced_dims["combined"] = (
    "Title: " + df_reduced_dims.Summary.str.strip() + "; Content: " + df_reduced_dims.Text.str.strip()
)

# Display the first 5 rows of the modified dataframe
df_reduced_dims.head(5)

In [ ]:

# Apply the `get_embedding` function to each entry in the 'combined' column and store the results
df_reduced_dims['embedding'] = df_reduced_dims['combined'].apply(lambda x: get_embedding_reduced_dims(x))

# Save the dataframe with embeddings to a CSV file, omitting the index
df_reduced_dims.to_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_reduced_dims_1k.csv', index=False)

# Display the first 5 rows of the modified dataframe
df.head(5)


In [ ]:

# Get the size of the data file
file_path = './EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_reduced_dims_1k.csv'
size_mb = get_file_size(file_path)
print(f"The size of the file is {size_mb:.2f} MB.")

## Question Answering Using Embeddings-Based Search
There are many common cases where the model is not trained on data which contains key facts and information you want to make accessible when generating responses to a user query. One way of solving this is to put additional information into the context window of the model. This is effective in many use cases but leads to higher token costs. The other way is to use RAG to obtain the information.

First, let's load up the file with the (large) embeddings in them.

In [ ]:

# Load the CSV file into a pandas DataFrame
df_search = pd.read_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_large_1k.csv')

# Convert the string representation of embeddings in the 'embedding' column to numpy arrays
# By converting the embeddings from string format to numpy arrays immediately after loading, 
# we ensure that the data is in a ready-to-use state for any subsequent analysis or processing steps.
df_search['embedding'] = df_search['embedding'].apply(eval).apply(np.array)

# Display the first 5 rows of the loaded dataframe
df_search.head(5)

### Relatedness
Now let's write some code to show the related reviews based on a ranking score. In this case, we will set up a function to take in our query string and compute the relateness scores for reviews. Then it will show us the top 5 revies in terms of relatedness score. 

In [ ]:

# Define a function that ranks strings from a pandas DataFrame based on their relatedness to a given query string.
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - distance.cosine(x, y), # use cosine similarity as the relatedness function
    top_n: int = 100,
    threshold: float = 0.001  # Minimum score difference to consider for ranking
) -> tuple[list[str], list[float]]:
    """
    Retrieve the top 'n' strings related to a query string from a DataFrame, based on a custom relatedness function.

    Parameters:
    query (str): The string to compare other strings against.
    df (pd.DataFrame): DataFrame containing the strings and their embeddings.
    relatedness_fn (callable): A function that computes the relatedness score between two embeddings. By default, 
                               it uses the cosine similarity between embeddings.
    top_n (int): The number of top related strings to return.
    threshold (float): The minimum difference between scores needed to consider one string more related than another.

    Returns:
    tuple[list[str], list[float]]: A tuple containing two lists:
                                   1. The top 'n' strings most related to the query.
                                   2. Their corresponding relatedness scores.
    """
    
    # Retrieve the embedding for the query string using the large model.
    query_embedding_response = client.embeddings.create(
        model="text-embedding-3-large",
        input=query,
    )
    # Extract the embedding data from the response.
    query_embedding = query_embedding_response.data[0].embedding

    # Compute the relatedness of each string in the DataFrame to the query string.
    strings_and_relatednesses = [
        (row["combined"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]

    # Sort the list of tuples by the relatedness score in descending order (most related first).
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)

    # Initialize a list to store the filtered results and a variable to track the last accepted score.
    filtered = []
    last_score = -1

    # Filter strings to meet the threshold criteria and limit the number of results to 'top_n'.
    for item in strings_and_relatednesses:
        if abs(item[1] - last_score) > threshold:
            filtered.append(item)
            last_score = item[1]
        if len(filtered) >= top_n:
            break

    # Unzip the tuples to separate strings and their relatedness scores.
    strings, relatednesses = zip(*filtered)

    # Return only the top 'n' results as specified by the function arguments.
    return strings[:top_n], relatednesses[:top_n]



In [ ]:
# This code performs a search to find the top 5 items related to the phrase "don't like the taste" 
# from a DataFrame 'df_search' using the previously defined 'strings_ranked_by_relatedness' function.

# Call the function 'strings_ranked_by_relatedness' with the query string, DataFrame, and specify the number 
# of top related items to return (top_n=5).
strings, relatednesses = strings_ranked_by_relatedness("don't like the taste", df_search, top_n=5)

# Loop over each string and its corresponding relatedness score.
# The 'zip' function combines the two lists 'strings' and 'relatednesses' so that items from both lists 
# can be accessed in a single loop iteration.
for string, relatedness in zip(strings, relatednesses):
    # Print a formatted string that includes a visual separator and the relatedness score formatted to three decimal places.
    print(f"\n========================\n{relatedness=:.3f}")
    
    # The 'display' function is typically used in Jupyter Notebooks or similar environments to render objects in a more
    # visually appealing manner than the basic print function. Here, it is used to display the string from the DataFrame.
    display(string)


## Asking Questions
Now that we have the ability to find reviews that are relevant we can ask questions based on the reviews retrieved and put into the context window. We will need a number of items to make sure we are getting good data and that we don't go over our token limit of 128k.

Let's start with a function to get a token count for any string passed into it.

In [ ]:

def num_tokens(text: str, model: str = 'gpt-4-turbo') -> int:
    """
    Return the number of tokens in a string using a specified model's tokenizer.

    Parameters:
        text (str): The input text to tokenize.
        model (str): The model whose tokenizer is used, defaulting to 'gpt-4-turbo'.

    Returns:
        int: The number of tokens that the text is divided into by the tokenizer.
    """

    # Get the tokenizer configuration specific to the model
    encoding = tiktoken.encoding_for_model(model)

    # Encode the text using the tokenizer and count the number of tokens
    return len(encoding.encode(text))


Next, let's create a function to pull reviews from our dataframe that can be put into the GPT context window

In [ ]:
def query_message(query: str, df: pd.DataFrame, model: str, token_budget: int) -> str:
    """
    Construct a message for GPT using related texts from a DataFrame, staying within a token limit.

    Parameters:
        query (str): The user's query to find related text.
        df (pd.DataFrame): DataFrame containing potential related texts.
        model (str): Model identifier for tokenization.
        token_budget (int): Maximum number of tokens allowed in the final message.

    Returns:
        str: A message constructed with introductory text, relevant reviews, and the query.
    """

    # Retrieve ranked strings and their relatedness from the dataframe
    strings, relatednesses = strings_ranked_by_relatedness(query, df)

    # Introduction to include in the message
    introduction = 'Use the below Amazon food reviews to answer the subsequent question. If the answer cannot be found in the reviews, write "I could not find an answer."'

    # Formatting the query into a question format
    question = f"\n\nQuestion: {query}"

    # Initialize the message with the introduction text
    message = introduction

    # Iterate over each related string to build the message
    for string in strings:
        # Format the string as an Amazon Food Review
        next_article = f'\n\nAmazon Food Review:\n"""\n{string}\n"""'

        # Check if adding the next article exceeds the token budget
        if num_tokens(message + next_article + question, model=model) > token_budget:
            break  # Stop adding articles if token budget is exceeded
        else:
            message += next_article  # Append the article if within budget

    # Return the final message with the question appended
    return message + question


Next, we create a function to actually allow us to ask questions from the GPT model

In [ ]:
def ask(
    query: str,
    df: pd.DataFrame,
    model: str = 'gpt-4-turbo',
    token_budget: int = 4096 - 500,
    print_message: bool = False
) -> str:
    """
    Generate an answer to a query using GPT, based on a DataFrame of related texts.

    Parameters:
        query (str): The query to be answered.
        df (pd.DataFrame): DataFrame containing texts and embeddings.
        model (str): Model identifier for tokenization, default is 'gpt-4-turbo'.
        token_budget (int): Maximum number of tokens for the generated message, default is 3596.
        print_message (bool): Whether to print the constructed message, default is False.

    Returns:
        str: The generated response from GPT based on the query and provided texts.
    """

    # Generate a message using related texts from the DataFrame
    message = query_message(query, df, model=model, token_budget=token_budget)

    # Optionally print the constructed message
    if print_message:
        print(message)

    # Set up the conversation format for the GPT model
    messages = [
        {"role": "system", "content": "You answer questions about Amazon Food Reviews."},
        {"role": "user", "content": message}
    ]

    # Request a completion from the GPT model
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0  # Use deterministic mode for reproducibility
    )

    # Extract the response message from the completion
    response_message = response.choices[0].message.content

    return response_message


Finally, let's ask our question

In [ ]:
# Submit our question to the model
ask("What are the top 3 reviews that indicate people don't like the food? Give me the title and text of the reviews in this format: Title: <title>; Content: <content>")

The output looks funky so let's pretty it up a bit. We can always try a variety of techniques, like outputting JSON, but, for now, we will just use regular expressions to deal with the output.

In [ ]:

def format_response(response: str) -> str:
    """
    Format a string containing multiple entries with titles and contents into a readable format.

    Parameters:
        response (str): The string response containing multiple formatted entries.

    Returns:
        str: A formatted string where each entry is separated and clearly labeled.
    """
    
    # Compile a pattern to match entries formatted as "Title: <title>; Content: <content>"
    pattern = re.compile(r"Title: (.*?); Content: (.*?)(?=\nTitle: |$)", re.S)
    
    # Extract all matches using the pattern
    matches = pattern.findall(response)
    
    # List to store formatted entries
    formatted_lines = []
    
    # Format each match with proper numbering and formatting
    for index, (title, content) in enumerate(matches, start=1):
        # Strip and replace newline characters in content
        content = content.strip().replace('\n', ' ')
        # Append formatted title and content to the list
        formatted_lines.append(f"{index}. Title: **{title.strip()}**")
        formatted_lines.append(f"   Content: {content}")
        formatted_lines.append('')  # Blank line for separation

    # Join all formatted lines into a single string
    return '\n'.join(formatted_lines)

# Example usage with a provided function response
response = "Title: Not tasty; Content: The food was bland and unenjoyable.\nTitle: Bad Service; Content: Waited 30 minutes to get served."
formatted_output = format_response(response)
print(formatted_output)




## Semantic Search
Now we will do straight searching for similarities using an algorithm called "cosine similarity" to determine how similar the words we are looking for are to words in the reviews.

First let's load data into our dataframe.

In [ ]:
import pandas as pd
import numpy as np



In [ ]:
# load data into our dataframe and turn the embeddings into numpy arrays
df_semantic_search = pd.read_csv('./EmbeddingsDemoAssets/fine_food_reviews_with_embeddings_large_1k.csv')

df_semantic_search["embedding"] = df_semantic_search.embedding.apply(literal_eval).apply(np.array)

### Cosine Similarity
Now let's use cosine similarity, like we did before, to get relatedness scores for reviews

In [ ]:
# This code performs a search to find the top 5 items related to the phrase "don't like the taste" 
# from a DataFrame 'df_search' using the previously defined 'strings_ranked_by_relatedness' function.

# Call the function 'strings_ranked_by_relatedness' with the query string, DataFrame, and specify the number 
# of top related items to return (top_n=5).
strings, relatednesses = strings_ranked_by_relatedness("don't like the taste", df_semantic_search, top_n=5)

# Loop over each string and its corresponding relatedness score.
# The 'zip' function combines the two lists 'strings' and 'relatednesses' so that items from both lists 
# can be accessed in a single loop iteration.
for string, relatedness in zip(strings, relatednesses):
    # Print a formatted string that includes a visual separator and the relatedness score formatted to three decimal places.
    print(f"\n========================\n{relatedness=:.3f}")
    
    # The 'display' function is typically used in Jupyter Notebooks or similar environments to render objects in a more
    # visually appealing manner than the basic print function. Here, it is used to display the string from the DataFrame.
    display(string)

### Eucleidean Distance
Now let's try Eucleidian Distance to see the difference

In [ ]:

def rank_strings_by_euclidean(query: str, df: pd.DataFrame, top_n: int = 100) -> tuple[list[str], list[float]]:
    """
    Returns a list of strings and their relatedness scores sorted from most related to least.
    The relatedness is determined using the Euclidean distance.
    
    Parameters:
        query (str): The query string to compare against dataframe strings.
        df (pd.DataFrame): DataFrame containing strings and their embeddings.
        top_n (int): The number of top related strings to return.

    Returns:
        tuple: A tuple containing two lists; one for strings and one for their corresponding relatedness scores.
    """

    # Retrieve the embedding for the query string from a model
    query_embedding_response = client.embeddings.create(
        model='text-embedding-3-large',
        input=query,
    )
    # Extract the embedding from the response
    query_embedding = query_embedding_response.data[0].embedding

    # Function to calculate negative Euclidean distance between two embeddings
    def calculate_relatedness(x, y):
        return -distance.euclidean(x, y)

    # Create a list of tuples containing strings from the DataFrame and their relatedness to the query
    strings_and_relatednesses = [
        (row["combined"], calculate_relatedness(query_embedding, row["embedding"]))
        for i, row in df.iterrows()  # Iterate over DataFrame rows
    ]

    # Sort the list of tuples by relatedness, descending
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)

    # Unzip the sorted list into two lists: one for strings and another for relatedness scores
    strings, relatednesses = zip(*strings_and_relatednesses)

    # Return the top N related strings and their scores
    return strings[:top_n], relatednesses[:top_n]


In [ ]:
# This code performs a search to find the top 5 items related to the phrase "don't like the taste" 
# from a DataFrame 'df_search' using the previously defined 'strings_ranked_by_relatedness' function.

# Call the function 'strings_ranked_by_relatedness_ed' with the query string, DataFrame, and specify the number 
# of top related items to return (top_n=5).
strings, relatednesses = strings_ranked_by_relatedness_ed("don't like the taste", df_semantic_search, top_n=5)

# Loop over each string and its corresponding relatedness score.
# The 'zip' function combines the two lists 'strings' and 'relatednesses' so that items from both lists 
# can be accessed in a single loop iteration.
for string, relatedness in zip(strings, relatednesses):
    # Print a formatted string that includes a visual separator and the relatedness score formatted to three decimal places.
    print(f"\n========================\n{relatedness=:.3f}")
    
    # The 'display' function is typically used in Jupyter Notebooks or similar environments to render objects in a more
    # visually appealing manner than the basic print function. Here, it is used to display the string from the DataFrame.
    display(string)

## Recommendation Example
Because shorter distances between embedding vectors represent greater similarity, embeddings can be useful for recommendation.

Below, we illustrate a basic recommender. It takes in a list of strings and one 'source' string, computes their embeddings, and then returns a ranking of the strings, ranked from most similar to least similar. As a concrete example, we apply a version of this function to the AG news dataset (sampled down to 2,000 news article descriptions) to return the top 5 most similar articles to any given source article.

### Load the Data
First, let's load our data into a dataframe

In [ ]:
# load data from dataset
# full dataset available at http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html
dataset_path = "./EmbeddingsDemoAssets/AG_news_samples.csv"
df = pd.read_csv(dataset_path)

# Number of examples to display
n_examples = 5

# Print the title, description, and label of the top n examples
for idx, row in df.head(n_examples).iterrows():
    print("\nTitle:", row['title'])
    print("Description:", row['description'])
    print("Label:", row['label'])


### Build a Cache to Save Embeddings
Before getting embeddings for these articles, let's set up a cache to save the embeddings we generate. In general, it's a good idea to save your embeddings so you can re-use them later. If you don't save them, you'll pay again each time you compute them again.

The cache is a dictionary that maps tuples of (text, model) to an embedding, which is a list of floats. The cache is saved as a Python pickle file.

In [ ]:

# Path to the embedding cache file
embedding_cache_path = "./EmbeddingsDemoAssets/recommendations_embeddings_cache.pkl"

# Try to load the embedding cache if it exists, otherwise initialize an empty dictionary
try:
    with open(embedding_cache_path, "rb") as embedding_cache_file:
        embedding_cache = pickle.load(embedding_cache_file)
except FileNotFoundError:
    embedding_cache = {}

# Define a function to retrieve embeddings from the cache or request via the API if not found
def embedding_from_string(string: str, model: str = "text-embedding-3-large", cache=embedding_cache) -> list:
    """
    Retrieve the embedding of a given string using a specified model.
    Utilizes a cache to avoid redundant API calls.

    Parameters:
        string (str): The string to embed.
        model (str): The model to use for embedding.
        cache (dict): The cache to store and retrieve embeddings.

    Returns:
        list: The embedding vector of the string.
    """
    # Check if the embedding is already in the cache
    if (string, model) not in cache:
        # Assume get_embedding_large is a function that requests the embedding from an API
        cache[(string, model)] = get_embedding_large(string, model)
        # Save the updated cache to disk
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(cache, embedding_cache_file)
    return cache[(string, model)]


Let's make sure everything is working by getting an embeddding

In [ ]:
# Example usage: Retrieving and displaying embeddings for descriptions from the dataset

# Fetch the first description from the dataset and print it
example_string = df["description"].iloc[0]  # Use iloc for safer access by index
print(f"\nExample string: {example_string}")

# Retrieve the embedding for the first description and print the first 10 dimensions
example_embedding = embedding_from_string(example_string)
print(f"\nExample embedding: {example_embedding[:10]}...")

# Fetch the tenth description from the dataset and print it
example_string_10 = df["description"].iloc[9]  # Use iloc for safer access by index
print(f"\nExample string: {example_string_10}")

# Retrieve the embedding for the tenth description and print the first 10 dimensions
example_embedding_10 = embedding_from_string(example_string_10)
print(f"\nExample embedding: {example_embedding_10[:10]}...")


### Recommend Similar Articles Based on Embeddings
To find similar articles, let's follow a three-step plan:

1. Get the similarity embeddings of all the article descriptions
2. Calculate the distance between a source title and all other articles
3. Print out the other articles closest to the source title

In [ ]:

def distances_from_embeddings(query_embedding, embeddings, distance_metric="cosine"):
    """
    Calculate distances from the query embedding to each embedding in the list.

    Parameters:
        query_embedding (array): The embedding of the query string.
        embeddings (list of arrays): A list of embeddings to compare against.
        distance_metric (str): The metric used to calculate distance, default is "cosine".

    Returns:
        list: Distances from the query embedding to each embedding in the list.
    """
    distances = []
    for emb in embeddings:
        dist = distance.cosine(query_embedding, emb) if emb is not None else np.inf
        distances.append(dist)
    return distances

def indices_of_nearest_neighbors_from_distances(distances, k=1):
    """
    Return indices of the k nearest neighbors based on the given distances.

    Parameters:
        distances (list): List of distances from the query.
        k (int): Number of nearest neighbors to return.

    Returns:
        array: Indices of the k nearest neighbors.
    """
    sorted_indices = np.argsort(distances)
    return sorted_indices[1:k+1]  # Skip the first index assuming it's the query itself

def print_recommendations_from_strings(strings, index_of_source_string, model='text-embedding-3-large', k_nearest_neighbors=1):
    """
    Print out the k nearest neighbors of a given string based on embeddings.

    Parameters:
        strings (list): List of strings to consider for neighbors.
        index_of_source_string (int): Index of the string to find neighbors for.
        model (str): Model used for generating embeddings.
        k_nearest_neighbors (int): Number of nearest neighbors to print.

    Returns:
        list: Indices of the nearest neighbors.
    """
    embeddings = [embedding_from_string(string, model=model) for string in strings]
    query_embedding = embeddings[index_of_source_string]
    distances = distances_from_embeddings(query_embedding, embeddings)
    indices_of_nearest_neighbors = indices_of_nearest_neighbors_from_distances(distances, k_nearest_neighbors + 1)

    query_string = strings[index_of_source_string]
    print(f"Source string: {query_string}")
    
    # Filter out the source string index and print recommendations
    recommendations = [i for i in indices_of_nearest_neighbors if i != index_of_source_string]
    for idx, rec_idx in enumerate(recommendations[:k_nearest_neighbors], 1):
        print(f"""
        --- Recommendation #{idx} (Nearest Neighbor #{idx} of {k_nearest_neighbors}) ---
        String: {strings[rec_idx]}
        Distance: {distances[rec_idx]:0.3f}
        """)

    return recommendations


In [ ]:
# NOTE: This code block may take a few minutes to run
article_descriptions = df["description"].tolist()

tony_blair_articles = print_recommendations_from_strings(
    strings=article_descriptions,  # let's base similarity off of the article description
    index_of_source_string=0,  # articles similar to the first one about Tony Blair
    k_nearest_neighbors=5,  # 5 most similar articles
)

In [ ]:
# NOTE: This code block may take a few minutes to run
chipset_security_articles = print_recommendations_from_strings(
    strings=article_descriptions,  # let's base similarity off of the article description
    index_of_source_string=1,  # let's look at articles similar to the second one about a more secure chipset
    k_nearest_neighbors=5,  # let's look at the 5 most similar articles
)


### Put The Results into JSON Format
JSON is a popular structured format that is used worldwide for transmitting text information. Let's put our Tony Blair articles in JSON format.

In [ ]:

# Create a list of dictionaries for each selected article, including available details
articles_json = [
    {
        'title': df.loc[index, 'title'],
        'description': df.loc[index, 'description']
    } for index in tony_blair_articles
]

# Optionally convert to a JSON string if needed for file storage or transmission
articles_json_string = json.dumps(articles_json, indent=4)

print(articles_json_string)
